<a href="https://colab.research.google.com/github/obeabi/SdsOctober2020Challenge/blob/main/Main_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Written by Abiola Obembe
# SDS October Challenge #2 - Used Car Prices
## 2021-01-12

### In this notebook a Neural Network is used to fit and predict used car price

This month you have been hired by a new car trader company to help sell its used cars. You will be analyzing used cars from multiple manufacturers and of different models. Your job is to help the car trader company determine the price of its used cars.

## Step 1: Data Preprocessing 

In [511]:
# Importing the libraries
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler 
import tensorflow as tf
tf.__version__
from sklearn.model_selection import train_test_split

%matplotlib inline
plt.rcParams['figure.figsize'] = (8.0, 6.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

print('Libraries installed successfully!')

np.random.seed(0)

Libraries installed successfully!


In [512]:
# Establish tensorflow version imported
tf.__version__

'2.4.0'

In [513]:
# Importing the training dataset

df_train = pd.read_csv('df_train_se2.csv', engine= 'python', encoding= 'latin1')

# drop first column
df_train.drop('Unnamed: 0', axis = 1, inplace = True)

df_train.head()

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed,price_usd,manufacturer_name,model_name,transmission,color,engine_fuel,engine_type,body_type,state,drivetrain
0,245000,2006,0,1.6,0,1,0,0,0,0,0,0,0,0,0,7,4250.0,20,612,1,1,3,2,2,2,1
1,265542,2002,0,3.3,0,0,1,0,0,0,0,0,0,0,0,133,4000.0,24,226,0,8,3,2,6,2,1
2,168000,2009,0,1.8,0,0,0,0,1,0,0,1,1,1,1,0,10900.0,20,395,1,1,0,0,6,2,1
3,225522,2008,0,1.8,0,0,1,1,0,0,1,0,0,1,1,20,6999.0,18,538,1,6,3,2,10,2,1
4,370000,1991,0,1.8,0,0,0,0,0,0,0,0,0,0,1,160,1600.0,29,101,1,0,3,2,8,2,1


In [514]:
# print shape and check for missing values
print(df_train.shape)
print(df_train.isnull().sum().sum())

(30824, 26)
0


In [515]:
# Importing the test dataset

df_test = pd.read_csv('df_test_se2.csv', engine= 'python', encoding= 'latin1')

# drop first column
df_test.drop('Unnamed: 0', axis = 1, inplace = True)

df_test.head()

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed,manufacturer_name,model_name,transmission,color,engine_fuel,engine_type,body_type,state,drivetrain
0,360000,1997,0,1.6,0,0,1,0,0,0,0,0,1,0,1,114,16,210,1,1,3,2,1,2,1
1,267000,1999,0,1.4,0,0,0,0,0,0,0,0,0,0,1,67,13,167,1,8,3,2,2,2,1
2,172000,2000,0,2.0,0,1,0,0,0,0,0,0,0,0,0,50,26,417,1,1,3,2,6,2,1
3,230000,2009,0,2.4,0,0,1,1,1,0,1,1,1,1,1,79,37,675,1,10,0,0,10,2,1
4,206000,2007,0,1.6,0,0,1,0,1,0,1,0,0,0,0,74,18,398,1,8,3,2,8,2,1


In [516]:
# print shape and number of total missing values
print(df_test.shape)
print(df_test.isnull().sum().sum())

(7707, 25)
0


In [517]:
# Let's create a function to create adjusted R-Squared
def adj_r2(x,y):
    r2 = regression.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [518]:
# Specify label column name here
label = 'price_usd'



# Rearrage the dataset columns
cols = df_train.columns.tolist()
colIdx = df_train.columns.get_loc(label)
# Do nothing if the label is in the 0th position
# Otherwise, change the order of columns to move label to 0th position
if colIdx != 0:
    cols = cols[colIdx:colIdx+1] + cols[0:colIdx] + cols[colIdx+1:]
    #cols =  cols[0:colIdx] + cols[colIdx:colIdx+1]
# Change the order of data so that label is in the 0th column
modified_df_train = df_train[cols]

modified_df_train.head()

,price_usd,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed,manufacturer_name,model_name,transmission,color,engine_fuel,engine_type,body_type,state,drivetrain
0,4250.0,245000,2006,0,1.6,0,1,0,0,0,0,0,0,0,0,0,7,20,612,1,1,3,2,2,2,1
1,4000.0,265542,2002,0,3.3,0,0,1,0,0,0,0,0,0,0,0,133,24,226,0,8,3,2,6,2,1
2,10900.0,168000,2009,0,1.8,0,0,0,0,1,0,0,1,1,1,1,0,20,395,1,1,0,0,6,2,1
3,6999.0,225522,2008,0,1.8,0,0,1,1,0,0,1,0,0,1,1,20,18,538,1,6,3,2,10,2,1
4,1600.0,370000,1991,0,1.8,0,0,0,0,0,0,0,0,0,0,1,160,29,101,1,0,3,2,8,2,1


In [519]:
# Check shape of bith train sets
print("Original test set shape is :" , df_train.shape)
print("Modified test set shape is :" , modified_df_train.shape)

Original test set shape is : (30824, 26)
Modified test set shape is : (30824, 26)


In [520]:
# Adjust test set also
test_list_columns = modified_df_train.columns[1:]

modified_df_test = df_test[test_list_columns]
modified_df_test.head()

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed,manufacturer_name,model_name,transmission,color,engine_fuel,engine_type,body_type,state,drivetrain
0,360000,1997,0,1.6,0,0,1,0,0,0,0,0,1,0,1,114,16,210,1,1,3,2,1,2,1
1,267000,1999,0,1.4,0,0,0,0,0,0,0,0,0,0,1,67,13,167,1,8,3,2,2,2,1
2,172000,2000,0,2.0,0,1,0,0,0,0,0,0,0,0,0,50,26,417,1,1,3,2,6,2,1
3,230000,2009,0,2.4,0,0,1,1,1,0,1,1,1,1,1,79,37,675,1,10,0,0,10,2,1
4,206000,2007,0,1.6,0,0,1,0,1,0,1,0,0,0,0,74,18,398,1,8,3,2,8,2,1


In [521]:
# Check shape of test set
print("Original test set shape is :" , df_test.shape)
print("Modified test set shape is :" , modified_df_test.shape)

Original test set shape is : (7707, 25)
Modified test set shape is : (7707, 25)


In [522]:
# Create X and y from train set
X = modified_df_train.iloc[:, 1:].values
y = modified_df_train.iloc[:, 0].values
X_test = modified_df_test.iloc[:,:].values
y = y.reshape(len(y),1)
print(X[0:1])

print("*"*30)
print(y[0:2])
print("*"*30)
print(X_test[0:1])

[[2.45e+05 2.01e+03 0.00e+00 1.60e+00 0.00e+00 1.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 7.00e+00
  2.00e+01 6.12e+02 1.00e+00 1.00e+00 3.00e+00 2.00e+00 2.00e+00 2.00e+00
  1.00e+00]]
******************************
[[4250.]
 [4000.]]
******************************
[[3.60e+05 2.00e+03 0.00e+00 1.60e+00 0.00e+00 0.00e+00 1.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 1.00e+00 0.00e+00 1.00e+00 1.14e+02
  1.60e+01 2.10e+02 1.00e+00 1.00e+00 3.00e+00 2.00e+00 1.00e+00 2.00e+00
  1.00e+00]]


In [523]:
# Splitting the dataset into the Training set and Validation set
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size = 0.08, random_state = 0)

In [524]:
# Feature Scaling
sc = MinMaxScaler(feature_range= (0,1))
sc_y = MinMaxScaler(feature_range= (0,1))

# Features columns
X_train = sc.fit_transform(X_train)
X_validation = sc.transform(X_validation)
X_test = sc.transform(X_test)

# target columns
y_train = sc_y.fit_transform(y_train)
y_validation = sc_y.transform(y_validation)


In [525]:
# Let's print X_train
print(X_train)

[[0.34 0.51 0.   ... 0.73 1.   0.5 ]
 [0.31 0.71 0.   ... 0.18 1.   0.5 ]
 [0.22 0.86 0.   ... 0.73 1.   0.5 ]
 ...
 [0.26 0.84 0.   ... 0.18 1.   0.5 ]
 [0.28 0.77 0.   ... 0.91 1.   0.5 ]
 [0.1  0.93 0.   ... 0.45 1.   0.5 ]]


## Building  ANN 

In [526]:
# Import keras libraries
from keras.layers import  Dropout,Flatten,Dense,Activation, BatchNormalization

In [527]:
# Initializing the ANN
ann = tf.keras.models.Sequential()

In [528]:
# Adding the input layer and the first hidden layer
ann.add(Dense(units=32, activation='relu'))
ann.add(BatchNormalization())
ann.add(Dropout(0.2))

In [529]:
# Adding the second hidden layer
ann.add(Dense(units=16, activation='relu'))
ann.add(BatchNormalization())
ann.add(Dropout(0.2))

In [530]:
# Adding the output layer
ann.add(Dense(units=1, activation=None))

## Training the ANN

In [531]:
# Compiling the ANN
ann.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Training the ANN on the Training set
ann.fit(X_train, y_train, batch_size = 2**5, epochs = 25,  validation_data=(X_validation, y_validation))

Epoch 1/25
887/887 [==============================] - 2s 2ms/step - loss: 0.4940 - val_loss: 0.0089
Epoch 2/25
887/887 [==============================] - 1s 2ms/step - loss: 0.0244 - val_loss: 0.0078
Epoch 3/25
887/887 [==============================] - 1s 2ms/step - loss: 0.0093 - val_loss: 0.0063
Epoch 4/25
887/887 [==============================] - 1s 2ms/step - loss: 0.0080 - val_loss: 0.0054
Epoch 5/25
887/887 [==============================] - 1s 2ms/step - loss: 0.0071 - val_loss: 0.0048
Epoch 6/25
887/887 [==============================] - 1s 2ms/step - loss: 0.0062 - val_loss: 0.0042
Epoch 7/25
887/887 [==============================] - 1s 2ms/step - loss: 0.0059 - val_loss: 0.0036
Epoch 8/25
887/887 [==============================] - 1s 2ms/step - loss: 0.0054 - val_loss: 0.0033
Epoch 9/25
887/887 [==============================] - 1s 2ms/step - loss: 0.0050 - val_loss: 0.0031
Epoch 10/25
887/887 [==============================] - 1s 2ms/step - loss: 0.0047 - val_loss: 0.0027

##  Making Predictions on ANN

In [532]:
# Make prediction on validation and test set
y_hat_validation = ann.predict(X_validation)
y_pred = ann.predict(X_test)
#y_hat_validation = sc_y.inverse_transform(y_hat_validation)
#y_validation = sc_y.inverse_transform(y_validation)
y_pred = sc_y.inverse_transform(y_pred)
np.set_printoptions(precision=2)
print(np.concatenate((y_hat_validation.reshape(len(y_hat_validation),1), y_validation.reshape(len(y_validation),1)),1))

[[0.25 0.18]
 [0.06 0.06]
 [0.4  0.48]
 ...
 [0.19 0.2 ]
 [0.08 0.09]
 [0.28 0.29]]


In [533]:
# Using scikit learn to evaluate performance
from sklearn.metrics import  r2_score, mean_squared_error
mse = mean_squared_error(y_hat_validation, y_validation)
print("The r2_score on validation set is :", r2_score(y_hat_validation, y_validation))
print("The mean squared error (MSE) on validation set is :", mse)
print("The root mean squared error (RMSE) on validation set is :", np.sqrt(mse))

The r2_score on validation set is : 0.8624905756570986
The mean squared error (MSE) on validation set is : 0.002180314813562663
The root mean squared error (RMSE) on validation set is : 0.04669384128086554


## # Conclusion: The model evaluation metrics on the validation set  are summarized as follows
* target encoding: r2_score = 0.84 , mse = 6013449 , rmse = 2452
* count encoding:  r2_score= 0.78 , mse = 6334479 , rmse = 2516 
* selective encoding:  r2_score= 0.77 , mse = 6359849, rmse = 2521
* selective encoding2 :  r2_score= 0.85 , mse = 5396513, rmse = 2323

### Save ANN Model

In [534]:
# Save model
ann.save("Carprice_20epochs.h5")